In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
import random
from tqdm.autonotebook import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, auc

np.random.seed(42)

c:\contests\chgk_irt_new\.venv\lib\site-packages\ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [2]:
with open('online.json') as f:
    online = json.load(f)
    online = pd.DataFrame(online).dropna()
    online.loc[:, 'IsOnline'] = 1
with open('offline.json') as f:
    offline = json.load(f)
    offline = pd.DataFrame(offline).dropna()
    offline.loc[:, 'IsOnline'] = 0

## Test/train split

In [3]:
on_train, on_test = train_test_split(online, test_size=0.3, random_state=42)
off_train, off_test = train_test_split(offline, test_size=0.3, random_state=42)
train = pd.concat([on_train, off_train], ignore_index=True)
test = pd.concat([on_test, off_test], ignore_index=True)

In [4]:
train.head(2)

,mask,TourId,teamIdList,teamId,teamName,IsOnline
0,000100001100100101111001011100111011,6728,"[59326, 110092, 167055, 110326, 221402]",51249,Два пальца в нёбо,1
1,111111001011110111101010100011110000011,6555,"[24593, 133148, 9485, 78282, 23386, 12631]",45365,Бахус crew,1


In [5]:
train.shape

(34430, 6)

In [6]:
offline.shape

(38591, 6)

In [24]:
test.shape

(14758, 6)

In [23]:
test.teamId.isin(train.teamId).sum()

13401

In [13]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
    
def at_least_same_taken_prob(probabilities, taken):
    ques_amount = len(probabilities)
    prob_matrix = np.zeros(shape=(ques_amount+1, ques_amount+1))
    prob_matrix[0, 0] = 1
    for i in range(ques_amount+1):
        for j in range(1, ques_amount+1):
            prob_matrix[i, j] = prob_matrix[i, j-1] * (1 - probabilities[j-1]) + \
                                prob_matrix[i-1, j-1] * probabilities[j-1]
    return prob_matrix[taken:, -1].sum()
    
class IRT_Model:
    def __init__(self, data):
        self.seed = 42
        self.init_value = 0.5
        self.prepared_data = self.prepare_data(data)
        self.init_question_difficulties()
        self.init_team_online_pref()
        self.init_players_skills()
        self.losses = list()
        self.rocaucs = list()
        self.probs = None
        pass
    
    def prepare_data(self, data):
        df = data.copy(deep=True)
        df.loc[:, 'Mask'] = df.loc[:, 'mask'].apply(lambda x: list(x))
        df = df.explode('Mask')
        df.loc[:, 'QuestionNumber'] = df.reset_index().groupby('index').cumcount().values
        df = df.sort_values(['TourId', 'QuestionNumber']).reset_index(drop=True)
        df.TourId = df.TourId.astype(int)
        df.QuestionNumber = df.QuestionNumber.astype(int)
        df = df.loc[df.Mask.isin(['0', '1']), :]
        df.Mask = df.Mask.astype(int)        
        df.loc[:, 'QuestionId'] = df.TourId * 1000 + df.QuestionNumber
        df.drop(['QuestionNumber', 'mask'], axis=1, inplace=True)
        return df
    
    def init_question_difficulties(self):
        # construc question difficulties df
        self.question_difficulties = pd.DataFrame(self.prepared_data.QuestionId.unique(),
                                                  columns=['QuestionId']).set_index('QuestionId')
        self.question_difficulties.loc[:, 'Difficulty'] = self.init_value
    
    def init_team_online_pref(self):
        # Construct team prefference to online df
        self.team_online_pref = pd.DataFrame(self.prepared_data.teamId.unique(),
                                                  columns=['TeamId']).sort_values('TeamId').set_index('TeamId')
        self.team_online_pref.loc[:, 'OnlinePref'] = self.init_value
    
    def init_players_skills(self):
        # Construct players' skills df
        self.players_skills = pd.DataFrame(self.prepared_data.teamIdList.explode().unique(),
                                           columns=['PlayerId']).sort_values('PlayerId').set_index('PlayerId')
        self.players_skills.loc[:, 'Skill'] = self.init_value
    
    def team_performance(self, players: list):
        # one possible measure of team performance
        return self.players_skills.loc[players, 'Skill'].mean()
        
    def fit(self, lr=0.01, epochs=10, batch_size=16):
        for i in tqdm(range(epochs)):
            batch = self.prepared_data.sample(n=batch_size)
            # calculate teams' performance
            team_performance = batch.loc[:, 'teamIdList'].apply(
                lambda members: self.team_performance(members)).values
            # get question difficulty
            q_dif = self.question_difficulties.loc[batch.loc[:, 'QuestionId'], 'Difficulty'].values
            # get team online preferrability
            team_online_pref = self.team_online_pref.loc[batch.loc[:, 'teamId'], 'OnlinePref'].values
            # get isOnline
            isOnline = batch.loc[:, 'IsOnline'].values
            
            # calculate  result 
            team_sigmoid = sigmoid(team_performance - q_dif + isOnline * team_online_pref)
            
            # update question difficulty
            q_dif_update = lr * (batch.loc[:, 'Mask'].values * (1-team_sigmoid) - 
                                 (1 - batch.loc[:, 'Mask'].values) * team_sigmoid)
            self.question_difficulties.loc[batch.loc[:, 'QuestionId'], 'Difficulty'] -= q_dif_update
            
            # update player skills
            skill_update = -lr * (batch.loc[:, 'Mask'].values * (1 - team_sigmoid) -
                                  (1 - batch.loc[:, 'Mask'].values) * team_sigmoid)
            skill_update_array = pd.DataFrame({0: batch.loc[:, 'teamIdList'].values, 1: skill_update}) \
                                   .explode(0).values
            self.players_skills.loc[skill_update_array[:, 0], 'Skill'] -= skill_update_array[:, 1]
            
            # update team prefferability to online (maybe should be player-related )
            online_pref_update = -lr * batch.loc[:, 'IsOnline'].values * \
                                 (batch.loc[:, 'Mask'].values * (1-team_sigmoid) - 
                                 (1 - batch.loc[:, 'Mask'].values) * team_sigmoid)
            self.team_online_pref.loc[batch.loc[:, 'teamId'].values, 'OnlinePref'] -= online_pref_update
            
            if i % 5 == 0:
                self.calculate_loss_and_rocauc()
            
        self.calculate_team_sigmoid()
            
    def calculate_team_sigmoid(self):
        self.prepared_data.loc[:, 'TeamPerformance'] = self.prepared_data.teamIdList.apply(self.team_performance)
        self.prepared_data.loc[:, 'TeamSigmoid'] = sigmoid(
            self.prepared_data.loc[:, 'TeamPerformance'].values - 
            self.question_difficulties.loc[self.prepared_data.loc[:, 'QuestionId'].values, 'Difficulty'].values +
            self.prepared_data.loc[:, 'IsOnline'].values *
            self.team_online_pref.loc[self.prepared_data.loc[:, 'teamId'].values, 'OnlinePref'].values
        )
    
    def calculate_loss_and_rocauc(self):
        self.calculate_team_sigmoid()
        tmp = self.prepared_data.copy(deep=True)
        tmp.loc[:, 'OneLoss'] = tmp.loc[:, 'Mask'].values * np.log(tmp.loc[:, 'TeamSigmoid'].values) + \
                                (1 - tmp.loc[:, 'Mask'].values) * np.log(1 - tmp.loc[:, 'TeamSigmoid'].values)
        total_loss = -tmp.loc[:, 'OneLoss'].values.sum()
        self.losses.append(total_loss)
        print(total_loss)
        # roc auc
        roc = roc_auc_score(tmp.loc[:, 'Mask'].values, tmp.loc[:, 'TeamSigmoid'].values)
        self.rocaucs.append(roc)
        print(roc)
        print('-'*15)
        
    def calculate_probs(self):
        if 'TeamSigmoid' not in self.prepared_data.columns:
            self.calculate_team_sigmoid()
        result = self.prepared_data.groupby(['TourId', 'teamId'])\
                 .apply(lambda row: 
                                 at_least_same_taken_prob(row['TeamSigmoid'].values, row['Mask'].sum()))\
                 .reset_index()\
                 .rename(columns={0: 'prob'})\
                 .sort_values('prob')\
                 .reset_index(drop=True)
        self.probs = result
    
        return result
        
    def get_test_instance(self, test_data):
        test = IRT_Model(test_data)
        # copy known question difficulties
        common_question_ids = test.question_difficulties.index.intersection(self.question_difficulties.index)
        test.question_difficulties.loc[:, 'Difficulty'] = self.question_difficulties.loc[:, 'Difficulty'].mean()
        test.question_difficulties.loc[common_question_ids, 'Difficulty'] = \
                                            self.question_difficulties.loc[common_question_ids, 'Difficulty']
        # copy known online prefference
        common_online_pref_ids = test.team_online_pref.index.intersection(self.team_online_pref.index)
        test.team_online_pref.loc[:, 'OnlinePref'] = self.team_online_pref.loc[:, 'OnlinePref'].mean()
        test.team_online_pref.loc[common_online_pref_ids, 'OnlinePref'] = \
                                            self.team_online_pref.loc[common_online_pref_ids, 'OnlinePref']
        # compy known skills
        common_skills_id = test.players_skills.index.intersection(self.players_skills.index)
        test.players_skills.loc[:, 'Skill'] = self.players_skills.loc[:, 'Skill'].mean()
        test.players_skills.loc[common_skills_id, 'Skill'] = self.players_skills.loc[common_skills_id, 'Skill']
        return test
        

In [15]:
irt = IRT_Model(train)

In [17]:
irt.fit(epochs=20, lr=0.4, batch_size=10000)

  0%|          | 0/20 [00:00<?, ?it/s]

897920.1901353
0.5974195419911875
---------------
826820.6083815583
0.7083027817479745
---------------
784494.317691766
0.7476664121299325
---------------
759193.5374721423
0.7673358720767708
---------------


In [18]:
irt.fit(epochs=20, lr=0.2, batch_size=10000)

  0%|          | 0/20 [00:00<?, ?it/s]

742992.026603494
0.7792225034099626
---------------
730583.3667156642
0.7892166933062028
---------------
720378.8998921954
0.7969554489012342
---------------
712313.5754853426
0.8026372172709251
---------------


In [19]:
irt.fit(epochs=30, lr=0.05, batch_size=10000)

  0%|          | 0/30 [00:00<?, ?it/s]

707098.6947665502
0.8060148868864684
---------------
704632.2481249991
0.8078236542100661
---------------
702294.7210925573
0.8095864042071934
---------------
699956.8526539801
0.8113421978000165
---------------
697819.041410895
0.8129140115596601
---------------
695812.3079701685
0.814321195853258
---------------


In [25]:
test_irt = irt.get_test_instance(test.loc[test.teamId.isin(train.teamId), :])

In [26]:
test_irt.calculate_loss_and_rocauc()

280646.43919767666
0.7966009160394709
---------------


In [21]:
import pickle

with open('irt_model.pkl', 'wb') as f:
    pickle.dump(irt, f, pickle.HIGHEST_PROTOCOL)

**Model**<br>
$p(y=1| S_{team}, \theta) = \sigma(S_{team} - \theta)$ <br>
$S_{team}$ - team skill<br>
$S_{team} = S_{player\_1} + S_{player\_2} + ... + S_{player\_N}$<br>
$\theta_j$ - difficulty of question j<br>
y - team answer, 1 for correct, 2 for incorrect<br>
$p(y_{hat}=y_{real}| S_{team}, \theta) = \sigma(S_{team} - \theta)^{y_{real}} * (1-\sigma(S_{team} - \theta))^{(1-y_{real})}$<br>
<br>
$LikelyHood = \frac{1}{N}\prod_{i=1}^{N} p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta)$<br>
$log(LikelyHood) = LL = \frac{1}{N}\sum_{i=1}^{N} log(p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta))=
y_{real}*log(\sigma(S_{team} - \theta)) + (1-y_{real})* log(1-\sigma(S_{team} - \theta))$<br>
<br>
$-log(LikelyHood) \rightarrow min$ w.r.t. $S_{player\_i}$ and $\theta_j$

**Model**<br>
$p(y=1| S_{team}, \theta, \gamma) = \sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)$ <br>
$S_{team}$ - team skill<br>
$S_{team} = S_{player\_1} + S_{player\_2} + ... + S_{player\_N}$<br>
$\theta_j$ - difficulty of question j<br>
$\gamma$ - the prefferabily of the team to online games
y - team answer, 1 for correct, 0 for incorrect<br>

$p(y_{hat}=y_{real}| S_{team}, \theta) = \sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)^{y_{real}} * (1-\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma))^{(1-y_{real})}$<br>
<br>

$LikelyHood = \frac{1}{N}\prod_{i=1}^{N} p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta, \gamma)$<br>

$log(LikelyHood) = LL = \frac{1}{N}\sum_{i=1}^{N} log(p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta, \gamma))=
y_{real}*log(\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)) + (1-y_{real})* log(1-\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma))$<br>
<br>

$-log(LikelyHood) \rightarrow min$ w.r.t. $S_{player\_i}$,  $\theta_j$, and $\gamma$

**Derivatives:**

$\sigma'(x) = \sigma(x)*(1-\sigma(x))$

$\frac{\partial LL}{\partial \theta} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) * (-1) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) * (-1) = 
-y_{real}*(1-\sigma(S_{team} - \theta)) + (1-y_{real})*\sigma(S_{team} - \theta)
$<br>

$\frac{\partial LL}{\partial S_i} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) = 
y_{real}*(1-\sigma(S_{team} - \theta)) - (1-y_{real})*\sigma(S_{team} - \theta)$<br>

**Derivatives:** NOT DONE!

$\sigma'(x) = \sigma(x)*(1-\sigma(x))$

$\frac{\partial LL}{\partial \theta} = \frac{y_{real}}{\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)} * \sigma'(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma) * (-1) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)} * (-\sigma'(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)) * (-1) = 
-y_{real}*(1-\sigma(S_{team} - \theta)) + (1-y_{real})*\sigma(S_{team} - \theta)
$<br>

$\frac{\partial LL}{\partial S_i} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) = 
y_{real}*(1-\sigma(S_{team} - \theta)) - (1-y_{real})*\sigma(S_{team} - \theta)$<br>

$\frac{\partial LL}{\partial \gamma_i} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) = 
y_{real}*(1-\sigma(S_{team} - \theta)) - (1-y_{real})*\sigma(S_{team} - \theta)$<br>

**Short derivatives:**

$\frac{\partial -LL}{\partial \theta} = +y_{real}*(1-\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)) - (1-y_{real})*\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)
$<br>

$\frac{\partial -LL}{\partial S_i} = -y_{real}*(1-\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)) + (1-y_{real})*\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)$<br>

$\frac{\partial -LL}{\partial \gamma} = +y_{real}* \mathrm{isOnline} * (1-\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)) - (1-y_{real}) * \mathrm{isOnline} *\sigma(S_{team} - \theta + \mathrm{isOnline} \cdot \gamma)
$<br>

**Gradient update**<br>
$S_{i\_new} = S_{i\_old} - learning\_rate * \frac{\partial LL}{\partial S_i}$<br>
$\theta_{new} = \theta_{old} - learning\_rate * \frac{\partial LL}{\partial \theta}$<br>